In [ ]:
!sudo pip3 install --upgrade pip keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow sklearn opencv-python optuna scikit-image  optkeras h5py==2.10.0

In [ ]:
!sudo pip3 install opencv-python

In [8]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile
import pickle
import pandas as pd

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *
# props = Properties()
# props["pegasus.mode"] = "development"
# props.write()

#Replica Catalog
rc = ReplicaCatalog()


input_csv = 'training_solutions_rev1.csv'
in_files=[]
    
input_files = [f for f in Path("./inputs").iterdir() if f.is_file() and f.name.endswith(".jpg")]
input_files.sort(key=lambda f: f.name.lower())
in_files=[]
 
for file in input_files:
    in_files.append(File(file.name))
    rc.add_replica(site="local", lfn=file.name, pfn=file.resolve())
rc.add_replica("local", File(input_csv), str(Path(".").resolve() / input_csv))  
rc.write()


#Transformation

data_label = Transformation( "Data_Label.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy/Data_Label.py",
            is_stageable=True
            )

pre_process_resize = Transformation( "Crop_Image.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy/Crop_Image.py",
            is_stageable=True
            )

pre_process_augment = Transformation( "Augment.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy/Augment.py",
            is_stageable=True
            )
       
tc = TransformationCatalog()\
    .add_transformations(data_label,pre_process_resize,pre_process_augment)\
    .write()

#Workflow
wf = Workflow("Galaxy", infer_dependencies=True)

dataset = []
for i in range(8):
    dataset.append(File('Class0_'+ str(i)+'.jpg'))
for i in range(8):
    dataset.append(File('Class1_'+ str(i)+'.jpg'))
for i in range(3):
    dataset.append(File('Class2_'+ str(i)+'.jpg'))
for i in range(4):
    dataset.append(File('Class3_'+ str(i)+'.jpg'))
for i in range(6):
    dataset.append(File('Class4_'+ str(i)+'.jpg'))
    
job_data_label = Job(data_label)\
                    .add_inputs(*in_files, File(input_csv))\
                    .add_outputs(*dataset)

resized_images = [File("resized_{}".format(f.lfn)) for f in dataset]
job_resize = Job(pre_process_resize)\
                    .add_inputs(*dataset)\
                    .add_outputs(*resized_images) 

Aug_images = []
for f in dataset:
    for i in range(1,5):
        Aug_images.append(File("Aug_{}_{}".format(os.path.splitext(str(f))[0],i) + '.jpg'))   
job_augment = Job(pre_process_augment)\
                    .add_inputs(*resized_images)\
                    .add_outputs(*Aug_images)


wf.add_jobs(job_data_label,job_resize,job_augment)                                    

In [9]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.12.09 23:42:38.388 UTC:
2020.12.09 23:42:38.400 UTC:   -----------------------------------------------------------------------
2020.12.09 23:42:38.414 UTC:   File for submitting this DAG to HTCondor           : Galaxy-0.dag.condor.sub
2020.12.09 23:42:38.429 UTC:   Log of DAGMan debugging messages                 : Galaxy-0.dag.dagman.out
2020.12.09 23:42:38.442 UTC:   Log of HTCondor libra

[#############-------------------------------------]  26.3% ..Running (Completed: 5, Queued: 0, Running: 1, Failed: 0)

KeyboardInterrupt: 